# HR Attrition Analysis

## Objective
Analyze who, when, and why attrition happens, forecast HR dynamics (5–10 years),
and optimize talent utilization.

## Tools Used
- LibreOffice: data cleaning
- SQL (SQLite): data extraction
- R: statistical modeling and forecasting
- Jupyter (Python): orchestration and reporting

In [2]:
import pandas as pd

df = pd.read_csv("../data_clean/hr_attrition_clean.csv")
df.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_Flag
0,41,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,Female,94,...,1,0,8,0,1,6,4,0,5,1
1,49,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,Male,61,...,4,1,10,3,3,10,7,1,7,0
2,37,Travel_Rarely,1373,Research & Development,2,2,Other,4,Male,92,...,2,0,7,3,3,0,0,0,0,1
3,33,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,Female,56,...,3,0,8,3,3,8,7,3,0,0
4,27,Travel_Rarely,591,Research & Development,2,1,Medical,1,Male,40,...,4,1,6,3,3,2,2,2,2,0


In [3]:
df.shape

(1470, 31)

In [4]:
df.columns

Index(['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Attrition_Flag'],
      dtype='object')

In [5]:
df["Attrition_Flag"].value_counts()

Attrition_Flag
0    1233
1     237
Name: count, dtype: int64

In [6]:
import sqlite3

conn = sqlite3.connect("../hr_attrition.db")

In [7]:
df.to_sql(
    "employee_attrition",
    conn,
    if_exists="replace",
    index=False
)

1470

In [8]:
pd.read_sql("SELECT COUNT(*) AS rows FROM employee_attrition", conn)

,rows
0,1470


In [9]:
pd.read_sql("""
SELECT 
    AVG(Attrition_Flag) AS overall_attrition_rate
FROM employee_attrition
""", conn)

,overall_attrition_rate
0,0.161224


In [10]:
pd.read_sql("""
SELECT 
    Department,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY Department
ORDER BY attrition_rate DESC
""", conn)

,Department,employees,attrition_rate
0,Sales,446,0.206
1,Human Resources,63,0.190
2,Research & Development,961,0.138


In [11]:
pd.read_sql("""
SELECT 
    JobRole,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY JobRole
ORDER BY attrition_rate DESC
""", conn)

,JobRole,employees,attrition_rate
0,Sales Representative,83,0.398
1,Laboratory Technician,259,0.239
2,Human Resources,52,0.231
3,Sales Executive,326,0.175
4,Research Scientist,292,0.161
5,Manufacturing Director,145,0.069
6,Healthcare Representative,131,0.069
7,Manager,102,0.049
8,Research Director,80,0.025


In [12]:
pd.read_sql("""
SELECT 
    Gender,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY Gender
""", conn)

,Gender,employees,attrition_rate
0,Female,588,0.148
1,Male,882,0.170


## Who Leaves – Key Observations
- Certain departments show higher attrition rates.
- Specific job roles are disproportionately affected.
- Gender differences are observable but require statistical validation.

In [13]:
pd.read_sql("""
SELECT 
    YearsAtCompany,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY YearsAtCompany
ORDER BY YearsAtCompany
""", conn)

,YearsAtCompany,employees,attrition_rate
0,0,44,0.364
1,1,171,0.345
2,2,127,0.213
3,3,128,0.156
4,4,110,0.173
5,5,196,0.107
6,6,76,0.118
7,7,90,0.122
8,8,80,0.113
9,9,82,0.098


In [14]:
pd.read_sql("""
SELECT 
    YearsSinceLastPromotion,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY YearsSinceLastPromotion
ORDER BY YearsSinceLastPromotion
""", conn)


,YearsSinceLastPromotion,employees,attrition_rate
0,0,581,0.189
1,1,357,0.137
2,2,159,0.170
3,3,52,0.173
4,4,61,0.082
5,5,45,0.044
6,6,32,0.188
7,7,76,0.211
8,8,18,0.000
9,9,17,0.235


In [15]:
pd.read_sql("""
SELECT 
    YearsInCurrentRole,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY YearsInCurrentRole
ORDER BY YearsInCurrentRole
""", conn)


,YearsInCurrentRole,employees,attrition_rate
0,0,244,0.299
1,1,57,0.193
2,2,372,0.183
3,3,135,0.119
4,4,104,0.144
5,5,36,0.028
6,6,37,0.054
7,7,222,0.140
8,8,89,0.079
9,9,67,0.090


## When Attrition Happens – Key Observations
- Attrition is concentrated in early tenure years.
- Lack of recent promotion correlates with higher attrition.
- Extended time in the same role increases exit risk.


In [16]:
pd.read_sql("""
SELECT 
    CASE
        WHEN MonthlyIncome < 3000 THEN 'Low Income'
        WHEN MonthlyIncome BETWEEN 3000 AND 7000 THEN 'Medium Income'
        ELSE 'High Income'
    END AS income_band,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY income_band
ORDER BY attrition_rate DESC
""", conn)


,income_band,employees,attrition_rate
0,Low Income,395,0.286
1,Medium Income,640,0.120
2,High Income,435,0.108


In [17]:
pd.read_sql("""
SELECT 
    OverTime,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY OverTime
""", conn)


,OverTime,employees,attrition_rate
0,No,1054,0.104
1,Yes,416,0.305


In [18]:
pd.read_sql("""
SELECT 
    JobSatisfaction,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY JobSatisfaction
ORDER BY JobSatisfaction
""", conn)


,JobSatisfaction,employees,attrition_rate
0,1,289,0.228
1,2,280,0.164
2,3,442,0.165
3,4,459,0.113


In [19]:
pd.read_sql("""
SELECT 
    WorkLifeBalance,
    COUNT(*) AS employees,
    ROUND(AVG(Attrition_Flag), 3) AS attrition_rate
FROM employee_attrition
GROUP BY WorkLifeBalance
ORDER BY WorkLifeBalance
""", conn)


,WorkLifeBalance,employees,attrition_rate
0,1,80,0.313
1,2,344,0.169
2,3,893,0.142
3,4,153,0.176


## Why Attrition Happens – Key Drivers
- Lower income bands show higher attrition.
- Overtime is strongly associated with exits.
- Low job satisfaction and poor work–life balance increase attrition risk.


In [20]:
df.to_csv("../outputs/hr_for_modeling.csv", index=False)

In [21]:
import pandas as pd

forecast = pd.read_csv("../outputs/attrition_forecast.csv")
model_coeffs = pd.read_csv("../outputs/attrition_model_coefficients.csv")

forecast

,Year,Expected_Attrition_Rate,Scenario
0,1,0.161,Baseline (no policy change)
1,2,0.161,Baseline (no policy change)
2,3,0.161,Baseline (no policy change)
3,4,0.161,Baseline (no policy change)
4,5,0.161,Baseline (no policy change)
5,6,0.161,Baseline (no policy change)
6,7,0.161,Baseline (no policy change)
7,8,0.161,Baseline (no policy change)
8,9,0.161,Baseline (no policy change)
9,10,0.161,Baseline (no policy change)


In [22]:
model_coeffs

,Unnamed: 0,Estimate,Std. Error,z value,Pr(>|z|)
0,(Intercept),1.356347,0.485063,2.796228,5.170285e-03
1,Age,-0.037258,0.010065,-3.701847,2.140357e-04
2,MonthlyIncome,-0.000078,0.000027,-2.921681,3.481478e-03
3,JobSatisfaction,-0.308056,0.068353,-4.506865,6.579256e-06
4,WorkLifeBalance,-0.266587,0.106003,-2.514902,1.190655e-02
5,YearsAtCompany,-0.083262,0.023907,-3.482795,4.962075e-04
6,YearsSinceLastPromotion,0.118542,0.035517,3.337625,8.449761e-04
7,OverTimeYes,1.457202,0.155465,9.373187,7.037500e-21


## Final Conclusions

### Who Leaves
- Attrition is concentrated in specific departments and job roles.
- Overtime employees show consistently higher exit rates.

### When Attrition Happens
- Highest risk occurs in early tenure years.
- Long periods without promotion increase attrition probability.

### Why Attrition Happens
- Low income, overtime, low job satisfaction, and poor work–life balance
  are key drivers of attrition.

### Forecast (5–10 Years)
- Baseline attrition is expected to remain stable without intervention.
- Optimized talent strategies can gradually reduce attrition over time.

### Talent Optimization Recommendations
- Reduce overtime dependency.
- Introduce targeted pay corrections for low-income bands.
- Implement structured promotion and role-rotation policies.
- Focus retention efforts on early-tenure employees.